In [1]:
import cv2
import numpy as np
import mediapipe as mp

#进度条库
from tqdm import tqdm
import time
import matplotlib.pyplot as plt

#使用ipython魔法方法，将绘制得到的图像直接嵌入在notebook单元格中
%matplotlib inline

In [2]:
seg = mp.solutions.selfie_segmentation.SelfieSegmentation(model_selection = 0)

## 处理单帧图像函数

In [3]:
import time
#定义逐帧处理函数，可不做任何处理，直接将摄像头捕捉到的画面写入视频帧
def process_frame(img):
    
    #记录该帧开始处理的时间
    start_time = time.time()
    
    
    img_RGB = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    bkgd_img = cv2.imread('./images/Camera3.jpg')

    # 将RGB图像输入模型，获取预测结果
    results = seg.process(img_RGB)

    mask = results.segmentation_mask.astype('uint8')

    mask_3 = np.stack((mask,mask,mask),axis = -1)
    mask_3 = mask_3 > 0.1

    # 从新背景图中切出原图大小的图块
    # 高度方向（Y方向）
    bottom = bkgd_img.shape[0]
    top = bottom - img.shape[0]

    # 宽度方向（x方向）
    left = bkgd_img.shape[1]//2-img.shape[1]//2  #从x方向中点向左偏移原始人像图片一半
    right = left + img.shape[1]

    new_bkgd = bkgd_img[top:bottom,left:right,:]
    assert new_bkgd.shape == img.shape,'新背景图片的长宽应大于原始图像长宽'

    ## 将mask_3定义的人像区加在新图块上
    # np.where(condition,A,B)  符合condition返回A，不符合返回B
    new_bkgd_mask_img = np.where(mask_3, img, new_bkgd)

    # 将完整背景对应的图块区域更换为加了人像的图块
    bkgd_img[top:bottom,left:right,:] = new_bkgd_mask_img
    
    # 记录该帧结束时间
    end_time = time.time()
    
     #计算每秒处理图像帧数FPS
    FPS = 1/(end_time - start_time)   #FPS大于30，成为实时目标检测算法
        
     #在图像上写FPS数值，参数依次为：图片、添加的文字、左上角坐标、字体、字体大小、颜色、字体粗细
    scaler = 1
    img = cv2.putText(bkgd_img,'FPS '+ str(int(FPS)), (25 * scaler,50 * scaler),cv2.FONT_HERSHEY_SIMPLEX,1.25 * scaler,(255,0,0),2 * scaler)
    
    return img

### 调用摄像头获取每帧

In [4]:
#调用摄像头拍摄视频模板
#生成的视频文件名默认为output_video.mp4，帧处理函数process_frame()默认不进行任何处理

import cv2
import time

output_name = 'record_video.mp4'

#获取摄像头
cap = cv2.VideoCapture(0)

#打开cap
cap.open(0)

frame_size = (cap.get(cv2.CAP_PROP_FRAME_WIDTH),cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
#fourcc = int(cap.get(cv2.CAP_PROP_FOURCC))
#fourcc = cv2.VideoWriter_fourcc(*'XVID')
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
fps = cap.get(cv2.CAP_PROP_FPS)

out = cv2.VideoWriter(output_name, fourcc=fourcc, fps=fps,frameSize=(int(frame_size[0]),int(frame_size[1])))


#无限循环，直到break被触发
while cap.isOpened():
    success,frame = cap.read()
    if not success:
        break
        
    #对捕捉到的帧进行图像处理
    frame = process_frame(frame)
    
    #展示处理后的二通道图像
    cv2.imshow('press q to break',frame)
    
    if cv2.waitKey(1) in [ord('q'),27]:    #按键盘上的q或esc退出（在英文输入法下）
        break
        
#关闭摄像头
cap.release()

#关闭图像窗口
cv2.destroyAllWindows()

print('视频已保存',output_name)

视频已保存 record_video.mp4
